In [ ]:
# Exercise 5.8: Racetrack (programming) Consider driving a race car around a
# turn like those shown in Figure 5.6. You want to go as fast as possible, but not so fast
# as to run off the track. In our simplified racetrack, the car is at one of a discrete set
# of grid positions, the cells in the diagram. The velocity is also discrete, a number of
# grid cells moved horizontally and vertically per time step. The actions are increments
# to the velocity components. Each may be changed by +1, −1, or 0 in one step, for a
# total of nine actions. Both velocity components are restricted to be nonnegative and
# less than 5, and they cannot both be zero except at the starting line. Each episode
# begins in one of the randomly selected start states with both velocity components
# zero and ends when the car crosses the finish line. The rewards are −1 for each step
# until the car crosses the finish line. If the car hits the track boundary, it is moved
# back to a random position on the starting line, both velocity components are reduced
# to zero, and the episode continues. Before updating the car’s location at each time
# step, check to see if the projected path of the car intersects the track boundary. If
# it intersects the finish line, the episode ends; if it intersects anywhere else, the car is
# considered to have hit the track boundary and is sent back to the starting line. To
# make the task more challenging, with probability 0.1 at each time step the velocity
# increments are both zero, independently of the intended increments. Apply a Monte
# Carlo control method to this task to compute the optimal policy from each starting
# state. Exhibit several trajectories following the optimal policy (but turn the noise
# off for these trajectories). 

In [2]:
import numpy as np

In [ ]:
#notes on track
#setting it up as dictionary where the coordinates are the keys and the values are what part of the track
    #track, off-track, start line, boundary
#for intersections, I'm tracing a hypothetical line on every movement from the start to the finish
    #I follow the path of that line in increments of 1
    #at every increment, check to see if in a reward or off-track square
    #if so then end the episode/restart the game, if not then continue along the line

In [5]:
#setting up the track
        
#track: consists of blocks featured plus an additional boundary boxes along the edges
#track 1
#17 blocks wide plus an additional left boundary for 18
#32 blocks high with an additional top boundary
#going cartesian coordinate stile where 0,0 is the lower left
#labeling the coordinates as: 0 for track, 1 for start line, 2 for off-track, 3 for reward line
track_1 = {}
for i in range(18):
    for j in range(32):
        if i == 0:
            track_1[(i,j)] = 2
        elif j == 31:
            track_1[(i,j)] = 2
        elif i == 1 and ( j < 18 or j > 26):
            track_1[(i,j)] = 2
        elif i == 2 and ( j < 10 or j > 27):
            track_1[(i,j)] = 2
        elif i == 3 and ( j < 3 or j > 29):
            track_1[(i,j)] = 2
        elif i > 9 and j < 25:
            track_1[(i,j)] = 2
        elif i > 10 and j < 26:
            track_1[(i,j)] = 2
        else:
            track_1[(i,j)] = 0

#starting line
starting_line = []
for i in range(4,10):
    starting_line.append((i,0))
    track_1[(i,0)] = 1
    
#finish line
#finish_line = []
for j in range(26,32):
    #finish_line.append((17,j))
    track_1[(17,j)] = 3

In [ ]:
#convenience functions for dealing with track

#get random starting coordinates
def random_start(start):
    return start[np.random.randint(0,len(start))]

#get type of track
def get_track_type(track,x,y):
    x = int(x)
    y = int(y)
    return track[(x,y)]

#get the track position where the car finished
# def get_finish_position(track,x,y,x_vel,y_vel):
#     return track[(x+x_vel,y+y_vel)]

#get spaces passed on movement
#not completely accurate as I'm moving over 1 coordinate at a time so could clip into off track
def boundary_track_passed(track,x_start,y_start,x_vel,y_vel):
    #move_list = [] #passing all the coordinates moved through to here
    
    #move_length = np.sqrt(x_vel**2 + y_vel**2)
    slope = y_vel/x_vel
    for i in range(1,x_vel+1):
        track_type = get_track_type(track,x_start+1,y_vel+slope*i)
        if track_type >= 2:
            return track_type
        
    return 0 #ie still on the board


In [19]:
#implement a MC algorithm
    #can do it like I solved cartpole with the Q table, take the current reward, update the state with the current reward and
        #the best possible future reward (iterated over possible actions)
    #many options:
        #on or off policy
        #exploring starts or not
        #first value or not
    #going to go with Monte Carlo ES from section 5.3

    
#convenience functions
def get_best_reward_and_action(q_table,pos,vel):
    temp_list = get_action_list(vel[0],vel[1])
    best_action = temp_list[0]
    best_reward = q_table[(pos,vel,temp_list[0])]
    
    for i in temp_list:
        temp_reward = q_table[(pos,vel,i)]
        if temp_reward > best_reward:
            best_reward = temp_reward
            best_action = i
            
    return best_reward, best_action
        

#can't increment below 0, can't have both be 0,0, neither can go above 5
def get_action_list(x_vel,y_vel):
    pi_a = []
    if x_vel + y_vel >= 1:
        pi_a.append(0,0)
        
        if x_vel < 5:
            pi_a.append(1,0)
        if y_vel < 5:
            pi_a.append(0,1)
        if x_vel < 5 and y_vel < 5:
            pi_a.append(1,1)
        
        if x_vel > 0 and y_vel > 0:
            pi_a.append(-1,-1)
        if x_vel > 0:
            pi_a.append(-1,0)
        if y_vel > 0:
            pi_a.append(0,-1)
    else 
        pi_a.append(1,0)
        pi_a.append(0,1)
        pi_a.append(1,1)

    return pi_a
    
#initialize Q(s,a) and pi(s)
Q = {} #value of state
pi_s = {} #best action found so far in that state
returns = {} #dictionary of empty lists

for key in track_1:
    track_type = track_1[key]
    if track_type >= 2:
        continue
    else if track_type == 1:
        pi_s[(key,(0,0))] = (1,1)
        #starting track
        x_vel = 0
        y_vel = 0
        pi_possible = get_action_list(x_vel,y_vel)
        for a in pi_possible:   
            Q[(key,(x_vel,y_vel),a)] = 0
            returns[(key,(x_vel,y_vel),a)] = []
        
    else:
        for x_vel in range(0,6):
            for y_vel in range(0,6):
                if i + j = 0:
                    continue
                pi_s[(key,(x_vel,y_vel))] = (0,0)
                else:
                    pi_possible = get_action_list(x_vel,y_vel)
                    
                for a in pi_possible:
                    Q[(key,(x_vel,y_vel),a)] = 0
                    returns[(key,(x_vel,y_vel),a)] = []
                    

In [ ]:
#unsure of what explicitly is meant by G. can do the current reward -1 for each step
#need to follow a policy so that all things equally likely
#could go random for a bit and see who it ends up but might take too long with calculations
episodes = 1000
explore_max = 0.95
explore_min = 0.5
steps = 0
gamma = 1

for i in range(episodes):
    start_pos = random_start(start_line)
    current_pos = start_pos
    x_vel = 0
    y_vel = 0
    #go on the track until finish line or reward
    on_track = True
    while on_track:
        steps += 1
        explore_p = explore_max - explore_max * steps/episodes + explore_min
        
        action_list = get_action_list(x_vel,y_vel)
        if np.random.rand() <= explore_p:
            #do a random action
            action = action_list[np.random.randint(0,len(action_list))]
        else:
            action = get_best_action()
        
        boundary_check = boundary_track_passed(track_1,current_pos[0],current_pos[1],action[0],action[1])
        if boundary_check == 3:
            #ends
            G = -1 + gamma*0
            rewards[(current_pos,(x_vel,y_vel),action)].append(G)
            break
        elif boundary_check == 2:
            #grab a new starting position
            next_pos = random_start(start_line)
        else:
            #still on the track
            next_pos = (current_pos[0] + action[0],current_pos[1]+action[1])
 
        #Getting returns G. uncertain how they want me to calculate G. ie how to use next state and discount factor
        next_reward, _ = get_best_reward_and_action(Q,next_pos,(x_vel,y_vel))
        G = -1 + gamma*next_reward 
        returns[(current_pos,(x_vel,y_vel),action)].append(G)
        
        #set Q(s,a)
        #Q is the average of the returns
        Q[(current_pos,(x_vel,y_vel),action)] = np.mean(returns[(current_pos,(x_vel,y_vel),action)])

        #set pi(s)
        _, best_action = get_best_reward_and_action(Q,next_pos,(x_vel,y_vel))
        pi[(current_pos,(x_vel,y_vel))] = best_action
        
        #setting the position and velocity for the next step
        current_pos = next_pos
        if boundary_check == 2:
            x_vel = 0
            y_vel = 0
        else:
            x_vel += action[0]
            y_vel += action[1]
            

In [ ]:
#print out the Q table
print(Q)